# Testing the Model Deployment

After deploying the model using RHOAI Model Serving, we'd like to test the model deployment by sending images to the model server for real-time inference.

In this notebook we'll review how to consume the model through the RHOAI Model Server.

We'll start by importing the preprocessing and rendering functions that we have worked with in the previous notebook.

In [1]:
import numpy as np

from requests import post
import torch

In [2]:
import numpy as np

loaded_array = np.load('data.npy')
print(loaded_array.shape)
print(loaded_array.dtype)


(1, 1, 160, 160, 160)
float32


Let's prepare one of our sample images as a test sample.

In [3]:
prediction_url = 'https://v98-spleen.apps.rhods-internal.61tk.p1.openshiftapps.com/v2/models/v98/infer'

We also need to know the class labels of the objects the model has been trained to detect. In case of the default YOLO v5 model, we can take the default class labels defined in the _classes_ module.
If you want to test a custom model, replace `default_class_labels` with the list of your custom class labels, e.g.

`['Laptop', 'Computer keyboard', 'Table']`.

In [4]:
filename = 'data.npy'
with open(filename, 'rb') as inputfile:
    mydata = np.load(inputfile)
mydata = mydata[:, :, 0:96, 0:96, 0:96]
mydata.shape

(1, 1, 96, 96, 96)

In [5]:
import numpy as np

# Create a large array of size 160x160x160 with example data
large_array = np.random.rand(160, 160, 160)  # Example with random values

# Extract the subarray of the first 96x96x96 elements
sub_array = large_array[0:96, 0:96, 0:96]

print(sub_array.shape)

(96, 96, 96)


In [12]:
image = mydata.flatten().tolist()
for i, element in enumerate(image):
    new_element = round(element, 2)
    image[i] = new_element

In [7]:
def serialize(image):
    payload = {
        'inputs': [
            {
                'name': 'x',
                'shape': [1, 1, 96, 96, 96],
                'datatype': 'FP32',
                'data': image,
            }
        ]
    }
    return payload

In [15]:
payload = serialize(image)

Let's now send the serialized image to the model server. The inference results will also be returned in a generic JSON structure, which we can unpack straightaway. We'll also apply the post-processing function we defined in the previous notebook to extract the familiar object properties.

In [9]:
def get_model_response(payload, prediction_url, classes_count):
    raw_response = post(prediction_url, json=payload)
    try:
        response = raw_response.json()
    except:
        print(f'Failed to deserialize service response.\n'
              f'Status code: {raw_response.status_code}\n'
              f'Response body: {raw_response.text}')
    try:
        model_output = response['outputs']
    except:
        print(f'Failed to extract model output from service response.\n'
              f'Service response: {response}')
    return model_output

In [16]:
response = get_model_response(payload, prediction_url, 0)
response

[{'name': '218',
  'datatype': 'FP32',
  'shape': [1, 2, 96, 96, 96],
  'data': [-0.22198644,
   -0.07239433,
   -0.20401585,
   0.2427446,
   -0.24878846,
   -0.24841452,
   -0.10708733,
   0.60254246,
   -0.3083517,
   0.18192673,
   -0.27506024,
   -0.15849246,
   -0.50619984,
   0.42123637,
   -0.30917972,
   0.22863655,
   -0.55714136,
   0.1757877,
   -0.44200838,
   0.0243313,
   -0.63878024,
   0.060369454,
   -0.41329736,
   0.21652399,
   -0.67763937,
   -0.03503006,
   -0.57623917,
   -0.028591555,
   -0.6331925,
   0.090638116,
   -0.3445603,
   0.009240508,
   -0.5392211,
   -0.055478536,
   -0.41836405,
   -0.04130677,
   -0.54443467,
   -0.039540425,
   -0.37194508,
   0.117025405,
   -0.66773605,
   -0.007683024,
   -0.5283095,
   0.016651655,
   -0.6396327,
   0.31426162,
   -0.3766091,
   -0.0040243724,
   -0.63858366,
   -0.068337105,
   -0.48507592,
   0.038165934,
   -0.5907099,
   0.045026034,
   -0.3677683,
   0.26498502,
   -0.5796812,
   0.176242,
   -0.4467765

Let's now visualize the result as in the previous notebook.

We were able to reproduce the object detection example from the previous notebook, so we can consume the deployed model as expected.

You can now head over to deploying the object detection application to consume this model in a real-time fashion.

After that, we'll explore offline scoring in the next notebook.